In [1]:
'''
model : BehradG/resnet-18-finetuned-MRI-Brain
resnet18 pretrained on brain mri images
model first layer modified to take 1-channel image instead of 3-channel
model last layer modified to output=1 instead of output=2
model fine tuned on just first layer and last layer 

'''

'\nmodel : BehradG/resnet-18-finetuned-MRI-Brain\nresnet18 pretrained on brain mri images\nmodel first layer modified to take 1-channel image instead of 3-channel\nmodel last layer modified to output=1 instead of output=2\nmodel fine tuned on just first layer and last layer \n\n'

In [2]:
import os

import numpy as np
import pandas as pd
# from sklearn.utils.class_weight import compute_class_weight       

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from config import Config, Device
from datasets import MRIDataset, BalancedMRIDataset
from models import ResNet18MRI
from train import Trainer_for_resnet18
from test_file import Tester    

/Users/fateme/Projects/iaaa-project/iaaa-venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = Device.device
print(device)

mps


In [4]:
data_path = os.path.join(os.getcwd(), "data")
labels_path = "train.csv"

batch_size = Config.batch_size
num_epochs = Config.num_epochs
learning_rate = Config.learning_rate
mean = Config.mean # mean of the entire datasaet
std = Config.std # std of the entire dataaset
image_size = 224

In [5]:
resclaed_mean = round(mean/255,4) # re-scale the actual mean
rescaled_std = round(std/255, 4) # re-scale the actual std

train_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

augment_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

test_transforms = transforms.Compose([
    # transforms.Lambda(lambda img: img.astype(np.float32)),
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

In [6]:

train_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='train',
    transform=train_transforms,
    augment_transform=augment_transforms,
    augment=True,
    max_slices=20
)

val_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='val',
    transform=test_transforms,
    max_slices=20
)

test_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='test',
    transform=test_transforms,
    max_slices=20
)

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=32)
test_dl = DataLoader(test_dataset, batch_size=32)

In [7]:
data_, label_ = next(iter(train_dl))
data_.size()

torch.Size([32, 20, 224, 224])

In [8]:
model = ResNet18MRI().to(device=device)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at BehradG/resnet-18-finetuned-MRI-Brain and are newly initialized because the shapes did not match:
- resnet.embedder.embedder.convolution.weight: found shape torch.Size([64, 3, 7, 7]) in the checkpoint and torch.Size([64, 1, 7, 7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
for param in model.parameters():
    param.requires_grad = True

In [10]:
# loss and optimizer

# criterion = nn.BCEWithLogitsLoss().to(device)
# criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [11]:
model_name = model.__class__.__name__
model_name

'ResNet18MRI'

In [12]:
trainer = Trainer_for_resnet18(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_dl=train_dl,
    val_dl=val_dl,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    num_epochs=100,
    patience=10,
    threshold=0.5,
    save_path=f"saved_models/{model_name}.pth"
)

# Start training
trainer.train()

100%|██████████| 89/89 [06:06<00:00,  4.12s/it]


Confusion Matrix:
[[10900    60]
 [ 1560     0]]
Epoch 1/100, Train Loss: 819.7164, Train Accuracy: 0.7755
Epoch 1/100, Val Accuracy: 0.8706, Precision: 0.0000, Recall: 0.0000, AUC: 0.4973, Avg Metric: 0.2902


100%|██████████| 89/89 [06:37<00:00,  4.47s/it]


Confusion Matrix:
[[9820 1140]
 [1460  100]]
Epoch 2/100, Train Loss: 803.9556, Train Accuracy: 0.7911
Epoch 2/100, Val Accuracy: 0.7923, Precision: 0.0806, Recall: 0.0641, AUC: 0.4800, Avg Metric: 0.3124


100%|██████████| 89/89 [06:39<00:00,  4.48s/it]


...

Confusion Matrix:
[[9600 1360]
 [1460  100]]
Epoch 29/100, Train Loss: 264.3314, Train Accuracy: 0.9230
Epoch 29/100, Val Accuracy: 0.7748, Precision: 0.0685, Recall: 0.0641, AUC: 0.4700, Avg Metric: 0.3025


100%|██████████| 89/89 [05:53<00:00,  3.97s/it]


Confusion Matrix:
[[10120   840]
 [ 1520    40]]
Epoch 30/100, Train Loss: 247.7530, Train Accuracy: 0.9387
Epoch 30/100, Val Accuracy: 0.8115, Precision: 0.0455, Recall: 0.0256, AUC: 0.4745, Avg Metric: 0.2942


100%|██████████| 89/89 [05:53<00:00,  3.97s/it]


Confusion Matrix:
[[8520 2440]
 [1320  240]]
Epoch 31/100, Train Loss: 225.3331, Train Accuracy: 0.9415
Epoch 31/100, Val Accuracy: 0.6997, Precision: 0.0896, Recall: 0.1538, AUC: 0.4656, Avg Metric: 0.3144
Early stopping triggered
